<img src = "https://image-src.bcg.com/Images/Zurich_76734213_2360x922_tcm58-177074.jpg" width = "383" align="center" >  </a>







<h1 align=left><font size = 5>Where should I live in Zurich? IBM Data Science Capstone Project</font></h1>
<h3 align=left><font size = 3>Author: Gian Fink</font></h3>

## ABSTRACT
This data science project is submitted in partial fulfillment for the IBM Data Science Professional Certificate. The goal of this project is to identify the five most attractive urban living areas for young people in Zurich. To do so, publicly available census data on neighbourhood level was combined with Foursquare data to run a machine learning clustering algorithm that identified the most attractive neighbourhoods. My findings suggest that young people who want to live in an urban neighbourhood in Zurich should find an apartment in (1) Höngg, (2) Seebach, (3) Wipkingen, (4) Hard or (5) Oerlikon. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Introduction </a>

2. <a href="#item2">Data</a>
   
3. <a href="#item3">Methodology</a>

4. <a href="#item4">Results</a>

5. <a href="#item5">Discussion</a>  
    
6. <a href="#item6">Conclusion</a> 
</font>
</div>

<a id='item10'></a>

# Week 1 Submission

<a id='item1'></a>

## 1. Introduction

This Section discusses the problem, the background and the relevance for specific target audience groups.

__Background:__ 

Zurich is the largest Swiss city and Switzerland's financial and economic capital. The city is located at the northwestern tip of Lake Zurich and as of January 2020, the municipality had 434'335 inhabitants. The consulting firm Mercer has ranked Zurich as the city with the highest living standard for several subsequent years (Mercer, 2007; Mercer, 2008). According to Mercer, Zurich has particularly high scores in the categories work, housing, leisure, education and safety.

Despite the high overall living standard, there are differences between the 34 neighbourhoods in Zurich. In urban downtown neighbourhoods you will usually find many bars and restaurants, whereas neighbourhoods in the outskrits of Zurich have less to offer. On the other hand, are they downtown neighbourhoods often noisier and more busy than neighbourhoods in the outskrits. Hence, there is somehow a tradeoff between enjoying the benefits of living downtown and enjoying the silence of outskirt neighbourhoods. 



__Problem Definition:__ 

This project aims to rank the top 5 urban neighbourhoods for young people in Zurich. In order to achieve this goal, three questions have to be asked: 
- Which data determines the attractiveness of a nighbourhood for young people?
- What are Zurich's urban neighbourhoods? What are Zurich's outskirts?
- Which of the urban neighbourhoods is most attractive for living?

__Targeted Audience:__

The results of this project might be relevant for people who intent to move to Zurich (e.g. experts, exchange students) or people who want to spend some holidays in Zurich. The results of this project might provide a starting point for finding an apartment in an attractive neighbourhood. This contribution is valuable insofar as moving to another city is often difficult because people lack knowledge about the quality of different neighbourhoods. 

<a id='item2'></a>

## 2. Data

This Section descripes the datasets that were included for this project. Three datasources will be incorporated for this project: 
- __Zurich Neighbourhoods Geo Location:__ As not structured list with Geo Location data for Zurich's neighbourshoods was found, data was gathered for each of the 34 neighbourhoods via GeoHack and stored in an Excel spreadsheet. This dataset included the variables: _district, quarter, latitude_ and _longitude_. (Source: https://tools.wmflabs.org/geohack/)
- __Census Data on Neighbourhood Level:__ The statistical office of Zurich provides detailed statistics for each neighbourhood. This data was downloaded and matched with the Zurich Neighbourhoods Geo Locations (see above). By that the additional data on the population and commercial density in each neighbourhood was obtained. (Source: https://www.stadt-zuerich.ch/prd/de/index/statistik/kreise-quartiere.html) 
- __Foursquare Venue Data:__ This data was used to cluster Zurich's neighbourhoods based on venue characteristics and a k-nearest algorithm. 

<a id='item3'></a>

## 3. Methodology

This Section describes the methodology including all required steps for this data science project.

__Import libraries:__ Before importing the different datasets, relevant libraries are installed. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import os

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Sol

__Import dataset:__ I have created a new neighourhood level (quarters) dataset based on the combination of data from different sources (see Section Data). I have stored this dataset as a .csv file on my computer and the following lines of code import this data as Pandas dataframe. 

In [2]:
# import data from .csv
df = pd.read_csv("zurich_data.csv", sep=';', delimiter=None, header='infer')
print("Data imported!")

Data imported!


In [3]:
# display data to check whether import worked
df.head()

,district,quarter,area_km2,latitude,longitude,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009
0,Kreis 1,Rathaus,0.38,47.38,8.55,10'199,1'588,4'179,8'703,0.48,3'307,3'267,3'266,3'226,3'197,3'236,3'194,3'164,3'140,3'099,3'110,30.6,30.1,30.5,30.6,30.2,29.7,29.0,28.8,28.6,28.8,28.7
1,Kreis 1,Hochschulen,0.56,47.38,8.55,14'033,734,1'311,1'211,1.08,678,664,677,640,641,634,665,673,681,678,673,31.7,34.3,32.6,32.5,31.8,29.8,30.7,30.8,28.9,28.9,30.8
2,Kreis 1,Lindenhof,0.23,47.37,8.54,13'851,1'194,5'191,4'387,1.18,1'009,990,955,972,974,955,923,935,950,951,940,29.0,30.1,28.7,27.3,27.2,27.4,25.8,26.4,24.9,25.0,23.4
3,Kreis 1,City,0.64,47.36,8.57,32'084,1'924,3'006,1'245,2.41,797,829,830,810,805,791,783,799,779,835,853,31.7,30.0,30.5,29.5,30.2,29.0,29.9,31.4,30.6,32.2,33.3
4,Kreis 2,Wollishofen,5.75,47.34,8.53,7'811,1'170,203,3'343,0.06,19'225,18'923,17'892,16'567,16'244,16'137,15'937,16'029,16'055,15'988,15'854,29.6,29.1,28.3,27.2,26.9,26.6,25.9,25.8,25.5,24.8,24.5


__Fetch geo coordinates from Zurich to create a map of Zurich:__

In [5]:
address = 'Zurich, ZH'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zurich are 47.3723941, 8.5423328.


__Create a map of Zurich with geo coordinates for each neighbourhood in Zurich:__

In [6]:
# create map of Zurich using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, district, quarter in zip(df.latitude, df.longitude, df.district, df.quarter):
    label = '{}, {}'.format(quarter, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  

    
map_zurich

__Foursquare Credentials:__

In [7]:
CLIENT_ID = 'ABBJUMT3NUSPKLEURXMV5SEGENGBO1SMXUE51F5WDSKTM2RS' # your Foursquare ID
CLIENT_SECRET = 'MPU1JRH1RPVEZBYFUXNEWVOV3MTJYEGOHTVRJP1UUDLAZX4J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ABBJUMT3NUSPKLEURXMV5SEGENGBO1SMXUE51F5WDSKTM2RS
CLIENT_SECRET:MPU1JRH1RPVEZBYFUXNEWVOV3MTJYEGOHTVRJP1UUDLAZX4J


__Explore the "Rathaus" quarter in Zurich:__

In [8]:
df.loc[0, 'quarter']

quarter_latitude = df.loc[0, 'latitude'] # quarter latitude value
quarter_longitude = df.loc[0, 'longitude'] # quarter longitude value

quarter_name = df.loc[0, 'quarter'] # quarter name



print('Latitude and longitude values of {} are {}, {}.'.format(quarter_name, 
                                                               quarter_latitude, 
                                                               quarter_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    quarter_latitude, 
    quarter_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


    
print('{} venues were returned for the "Rathaus" quarter by Foursquare.'.format(nearby_venues.shape[0]))


Latitude and longitude values of Rathaus are 47.38, 8.55.
21 venues were returned for the "Rathaus" quarter by Foursquare.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:48: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


__Explore all the 34 quarters in Zurich:__

_The code below retrieves Foursquare venues for all of the 34 quaters in Zurich:_

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['quarter', 
                  'latitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

_This code retrieves the Foursquare venues for each neighbourhood in Zurich:_

In [10]:
zurich_venues = getNearbyVenues(names=df['quarter'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Rathaus
Hochschulen
Lindenhof
City
Wollishofen
Leimbach
Enge
Alt-Wiedikon
Friesenberg
Sihlfeld
Werd
Langstrasse
Hard
Gewerbeschule
Escher Wyss
Unterstrass
Oberstrass
Fluntern
Hottingen
Hirslanden
Witikon
Seefeld
Mühlebach
Weinegg
Albisrieden
Altstetten
Höngg
Wipkingen
Affoltern
Oerlikon
Seebach
Saatlen
Schwamedingen-Mitte
Hirzenbach


_The table below shows for each neighbourhood in Zurich it's Foursquare venues:_

In [11]:
print(zurich_venues.shape)
zurich_venues.head()

(939, 7)


,quarter,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rathaus,47.38,8.55,The Alehouse - Palmhof,47.379138,8.548037,Gastropub
1,Rathaus,47.38,8.55,K55 GmbH,47.378553,8.548116,Electronics Store
2,Rathaus,47.38,8.55,Kleine Freiheit,47.379216,8.544588,Snack Place
3,Rathaus,47.38,8.55,Restaurant Haldenbach,47.379901,8.547052,Restaurant
4,Rathaus,47.38,8.55,"focusTerra, ETH Zürich",47.378231,8.547438,Science Museum


_Let's count the venues in each neighbourhood:_

In [12]:
zurich_venues.groupby('quarter').count()

,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
quarter,,,,,,
Affoltern,12,12,12,12,12,12
Albisrieden,1,1,1,1,1,1
Alt-Wiedikon,27,27,27,27,27,27
Altstetten,6,6,6,6,6,6
City,9,9,9,9,9,9
Enge,32,32,32,32,32,32
Escher Wyss,88,88,88,88,88,88
Fluntern,10,10,10,10,10,10
Friesenberg,12,12,12,12,12,12


_How many unique venue categories are overall in Zurich?_

In [13]:
print('There are {} uniques categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 135 uniques categories.


__Let's have a look at the venues of each Zurich neighbourhood:__

In [14]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")

# add quarter column back to dataframe
zurich_onehot['quarter'] = zurich_venues['quarter'] 

# move quarter column to the first column
fixed_columns = [zurich_onehot.columns[-1]] + list(zurich_onehot.columns[:-1])
zurich_onehot = zurich_onehot[fixed_columns]

zurich_onehot.head()

,quarter,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Garden,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Burger Joint,Bus Station,Business Service,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pizza Place,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Science Museum,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tennis Court,Thai Restaurant,Theater,Trade School,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rathaus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rathaus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rathaus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Rathaus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Rathaus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# What is the new dataframes size?
zurich_onehot.shape

(939, 136)

In [16]:
# This code groups the rows by quarter (neighbourhoods) and computes the mean frequency for each category.
zurich_grouped = zurich_onehot.groupby('quarter').mean().reset_index()
zurich_grouped.head()

,quarter,Accessories Store,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Garden,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Burger Joint,Bus Station,Business Service,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Historic Site,History Museum,Home Service,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Optical Shop,Other Great Outdoors,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pizza Place,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Sandwich Place,Science Museum,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tennis Court,Thai Restaurant,Theater,Trade School,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Affoltern,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.083333,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.083333,0.000000,0.0,0.0,0.0,0.000000,0.0
1,Albisrieden,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,Alt-Wiedikon,0.0,0.0,0.0,0.0,0.037037,0.000000,0.0,0.0,0.0,0.037037,0.037037,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.148148,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.037037,0.037037,0.0,0.037037,0.0,0.0,0.074074,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.074074,0.037037,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.074074,0.0,0.03703

In [17]:
zurich_grouped.shape

(34, 136)

__This code shows the top 5 venues for each Zurich neighbourhood:__

In [18]:
num_top_venues = 5

for hood in zurich_grouped['quarter']:
    print("----"+hood+"----")
    temp = zurich_grouped[zurich_grouped['quarter'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Affoltern----
                venue  freq
0         Bus Station  0.17
1         Supermarket  0.17
2    Department Store  0.08
3          Bike Trail  0.08
4  Italian Restaurant  0.08


----Albisrieden----
                   venue  freq
0                  Trail   1.0
1  Outdoors & Recreation   0.0
2                 Museum   0.0
3            Music Store   0.0
4            Music Venue   0.0


----Alt-Wiedikon----
                venue  freq
0                Café  0.15
1  Italian Restaurant  0.07
2    Swiss Restaurant  0.07
3   French Restaurant  0.07
4         Event Space  0.04


----Altstetten----
                       venue  freq
0                Bus Station  0.33
1                Pizza Place  0.17
2  Middle Eastern Restaurant  0.17
3               Soccer Field  0.17
4               Home Service  0.17


----City----
                venue  freq
0    Swiss Restaurant  0.22
1              Museum  0.11
2           Wine Shop  0.11
3  Italian Restaurant  0.11
4   French Restaurant  0.11



__This codes puts the data above into a Pandas dataframe:__

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['quarter']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
quarter_venues_sorted = pd.DataFrame(columns=columns)
quarter_venues_sorted['quarter'] = zurich_grouped['quarter']

for ind in np.arange(zurich_grouped.shape[0]):
    quarter_venues_sorted.iloc[ind, 1:] = return_most_common_venues(zurich_grouped.iloc[ind, :], num_top_venues)

quarter_venues_sorted

,quarter,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Affoltern,Supermarket,Bus Station,Department Store,Hotel,Italian Restaurant,Train Station,Athletics & Sports,Diner,Bike Trail,Light Rail Station
1,Albisrieden,Trail,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant
2,Alt-Wiedikon,Café,Swiss Restaurant,French Restaurant,Italian Restaurant,Pizza Place,Restaurant,Event Space,Ethiopian Restaurant,Japanese Restaurant,Bar
3,Altstetten,Bus Station,Middle Eastern Restaurant,Home Service,Soccer Field,Pizza Place,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Food Court
4,City,Swiss Restaurant,Mediterranean Restaurant,Italian Restaurant,Museum,Grocery Store,French Restaurant,Tram Station,Wine Shop,Department Store,Dessert Shop
5,Enge,Italian Restaurant,Park,Bar,Supermarket,Restaurant,Hotel,Tram Station,History Museum,Swiss Restaurant,Café
6,Escher Wyss,Bar,Café,Gym / Fitness Center,Clothing Store,Restaurant,Italian Restaurant,Hotel,Nightclub,Falafel Restaurant,Hotel Bar
7,Fluntern,Bakery,Tram Station,Gastropub,Bus Station,Supermarket,Grocery Store,Pizza Place,Historic Site,Hawaiian Restaurant,Cultural Center
8,Friesenberg,Supermarket,Indian Restaurant,Restaurant,Bus Station,Lounge,Beer Garden,Sandwich Place,Cheese Shop,Diner,Tram Station
9,Gewerbeschule,Bar,Italian Restaurant,Café,Thai Restaurant,Swiss Restaurant,Asian Restaurant,Bakery,Mediterranean Restaurant,Chinese Restaurant,Supermarket


__Let's cluster Zurich's neighbourhoods based on their venues on Foursquare:__

In [21]:
# set number of clusters
kclusters = 5

zurich_grouped_clustering = zurich_grouped.drop('quarter', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(zurich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 3, 0, 0, 0, 0, 0], dtype=int32)

In [22]:
# add clustering labels
quarter_venues_sorted.insert(0, 'labels', kmeans.labels_)

zurich_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
zurich_merged = zurich_merged.join(quarter_venues_sorted.set_index('quarter'), on='quarter')

zurich_merged.head() # check the last columns!

,district,quarter,area_km2,latitude,longitude,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Kreis 1,Rathaus,0.38,47.38,8.55,10'199,1'588,4'179,8'703,0.48,3'307,3'267,3'266,3'226,3'197,3'236,3'194,3'164,3'140,3'099,3'110,30.6,30.1,30.5,30.6,30.2,29.7,29.0,28.8,28.6,28.8,28.7,0,Hotel,Italian Restaurant,Bakery,Restaurant,Swiss Restaurant,Snack Place,Middle Eastern Restaurant,Gastropub,Electronics Store,Supermarket
1,Kreis 1,Hochschulen,0.56,47.38,8.55,14'033,734,1'311,1'211,1.08,678,664,677,640,641,634,665,673,681,678,673,31.7,34.3,32.6,32.5,31.8,29.8,30.7,30.8,28.9,28.9,30.8,0,Hotel,Italian Restaurant,Bakery,Restaurant,Swiss Restaurant,Snack Place,Middle Eastern Restaurant,Gastropub,Electronics Store,Supermarket
2,Kreis 1,Lindenhof,0.23,47.37,8.54,13'851,1'194,5'191,4'387,1.18,1'009,990,955,972,974,955,923,935,950,951,940,29.0,30.1,28.7,27.3,27.2,27.4,25.8,26.4,24.9,25.0,23.4,0,Swiss Restaurant,Hotel,Café,Bar,French Restaurant,Restaurant,Cocktail Bar,Wine Bar,Lounge,Plaza
3,Kreis 1,City,0.64,47.36,8.57,32'084,1'924,3'006,1'245,2.41,797,829,830,810,805,791,783,799,779,835,853,31.7,30.0,30.5,29.5,30.2,29.0,29.9,31.4,30.6,32.2,33.3,3,Swiss Restaurant,Mediterranean Restaurant,Italian Restaurant,Museum,Grocery Store,French Restaurant,Tram Station,Wine Shop,Department Store,Dessert Shop
4,Kreis 2,Wollishofen,5.75,47.34,8.53,7'811,1'170,203,3'343,0.06,19'225,18'923,17'892,16'567,16'244,16'137,15'937,16'029,16'055,15'988,15'854,29.6,29.1,28.3,27.2,26.9,26.6,25.9,25.8,25.5,24.8,24.5,0,Thai Restaurant,Irish Pub,Bus Station,Restaurant,Supermarket,Cheese Shop,Electronics Store,Fast Food Restaurant,Falafel Restaurant,Event Space


__Visualization of the Zurich neighbourhood clusters:__

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(zurich_merged.latitude, zurich_merged.longitude, zurich_merged.quarter, zurich_merged.labels):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
# Cluster 1
cluster1 = zurich_merged.loc[zurich_merged['labels'] == 0, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]
cluster1.sort_values(by=['wp_pop_ratio'])

,quarter,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Saatlen,1'516,236,209,7'824,0.03,8'841,8'582,8'388,8'283,8'508,7'563,7'280,7'118,7'131,7'175,7'132,27.9,28.8,30.2,30.8,31.2,31.0,31.0,31.9,32.2,31.8,32.7,0,Pool,Supermarket,Bus Station,Lounge,Bagel Shop,Dessert Shop,Diner,Eastern European Restaurant,Electronics Store,French Restaurant
28,Affoltern,4'106,911,151,4'422,0.03,26'710,26'562,26'177,26'054,25'874,25'902,25'082,24'855,24'437,22'972,22'383,32.7,33.0,32.9,32.8,33.1,33.5,32.5,32.5,32.1,31.4,31.7,0,Supermarket,Bus Station,Department Store,Hotel,Italian Restaurant,Train Station,Athletics & Sports,Diner,Bike Trail,Light Rail Station
8,Friesenberg,5'177,406,79,2'157,0.04,11'107,10'933,10'860,11'002,10'698,10'695,10'696,10'986,10'622,10'596,11'003,18.5,18.3,18.5,19.5,19.4,19.8,19.6,21.0,20.6,20.1,23.5,0,Supermarket,Indian Restaurant,Restaurant,Bus Station,Lounge,Beer Garden,Sandwich Place,Cheese Shop,Diner,Tram Station
26,Höngg,9'000,1'126,161,3'490,0.05,24'358,24'020,23'797,23'423,22'320,21'826,21'581,21'537,21'323,21'179,21'294,25.0,25.1,25.3,24.7,23.4,23.1,22.8,22.3,22.3,21.7,21.2,0,Tram Station,Plaza,Grocery Store,Pizza Place,Bus Station,Fast Food Restaurant,Mexican Restaurant,Sporting Goods Shop,Supermarket,Gas Station
30,Seebach,29'652,1'552,329,5'467,0.06,25'806,25'568,25'817,25'198,24'991,24'431,24'008,23'310,22'255,22'037,21'489,39.2,39.3,39.1,38.4,38.1,38.0,37.2,36.5,36.1,35.6,35.9,0,Hookah Bar,Supermarket,Bakery,Grocery Store,Korean Restaurant,Laser Tag,Pool,Restaurant,Plaza,Tram Station
4,Wollishofen,7'811,1'170,203,3'343,0.06,19'225,18'923,17'892,16'567,16'244,16'137,15'937,16'029,16'055,15'988,15'854,29.6,29.1,28.3,27.2,26.9,26.6,25.9,25.8,25.5,24.8,24.5,0,Thai Restaurant,Irish Pub,Bus Station,Restaurant,Supermarket,Cheese Shop,Electronics Store,Fast Food Restaurant,Falafel Restaurant,Event Space
27,Wipkingen,7'402,1'154,547,7'808,0.07,16'474,16'321,16'544,16'141,15'879,15'717,15'835,15'791,15'869,15'700,15'645,27.4,28.3,28.7,28.4,28.8,29.3,30.0,30.5,30.1,29.7,30.4,0,Bar,Café,Gym / Fitness Center,Clothing Store,Restaurant,Italian Restaurant,Hotel,Nightclub,Falafel Restaurant,Hotel Bar
12,Hard,6'954,961,658,8'945,0.07,13'060,13'163,13'057,12'891,13'072,13'232,13'241,12'994,12'744,12'883,12'902,36.2,37.4,38.0,38.6,38.4,39.9,39.9,40.3,40.6,41.0,42.6,0,Swiss Restaurant,Tram Station,Café,Restaurant,Italian Restaurant,Bar,Diner,Gym / Fitness Center,Peruvian Restaurant,Sports Bar
29,Oerlikon,23'553,1'858,696,8'722,0.08,23'288,23'214,22'815,22'161,21'966,21'895,21'658,21'331,21'253,20'787,20'740,38.3,38.3,38.8,38.0,37.2,37.1,36.3,35.7,35.7,34.7,34.2,0,Hookah Bar,Supermarket,Bakery,Grocery Store,Korean Restaurant,Laser Tag,Pool,Restaurant,Plaza,Tram Station
25,Altstetten,43'417,2'670,357,4'590,0.08,34'285,33'461,32'603,31'724,32'003,31'486,31'115,31'438,31'381,30'659,29'845,36.2,35.9,35.8,35.3,35.3,35.4,35.4,36.1,36.3,35.6,36.1,0,Bus Station,Middle Eastern Restaurant,Home Service,Soccer Field,Pizza Place,Department Store,Dessert Shop,Diner,Eastern European Restaurant,Food Court


__Finally let's have a look at the venues of each Zurich neighbourhood cluster:__

In [25]:
# Cluster 2
cluster2 = zurich_merged.loc[zurich_merged['labels'] == 1, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]
cluster2.sort_values(by=['wp_pop_ratio'])


,quarter,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Albisrieden,8'772,1'104,240,4'859,0.05,22'352,22'304,22'113,21'174,19'325,19'199,19'146,18'999,18'432,17'835,17'675,26.8,27.1,27.2,26.5,25.9,25.6,25.6,25.4,25.2,24.9,24.8,1,Trail,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Eastern European Restaurant


In [26]:
# Cluster 3
cluster3 = zurich_merged.loc[zurich_merged['labels'] == 2, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]
cluster3.sort_values(by=['wp_pop_ratio'])

,quarter,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Hirzenbach,2'229,316,121,5'011,0.02,13'129,12'801,12'627,12'463,11'930,11'679,11'153,11'404,11'516,11'459,11'610,35.5,35.7,35.9,36.5,36.3,36.0,35.7,35.2,35.1,34.5,36.0,2,Tram Station,Soccer Field,Steakhouse,Baseball Field,Ethiopian Restaurant,Food Court,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space
32,Schwamedingen-Mitte,3'293,514,230,5'050,0.05,11'261,11'100,11'012,11'076,11'315,11'301,11'209,10'934,10'863,10'903,10'857,42.7,42.1,42.3,41.6,41.8,41.2,40.4,39.7,39.4,38.8,39.3,2,Tram Station,Pizza Place,Italian Restaurant,Eastern European Restaurant,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [27]:
# Cluster 4
cluster4 = zurich_merged.loc[zurich_merged['labels'] == 3, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]
cluster4.sort_values(by=['wp_pop_ratio'])

,quarter,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Witikon,1'878,519,105,2'242,0.05,11'054,10'953,10'600,10'667,10'639,10'406,10'267,10'246,10'258,10'242,10'284,26.6,26.0,25.3,25.2,24.9,24.1,23.5,22.5,21.8,21.0,20.8,3,Cosmetics Shop,Swiss Restaurant,Other Great Outdoors,Ethiopian Restaurant,Food Court,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Yoga Studio
19,Hirslanden,3'520,757,344,3'410,0.10,7'503,7'488,7'465,7'321,7'380,7'403,7'285,7'131,7'024,6'998,6'956,26.9,27.5,27.2,26.6,27.3,27.5,26.2,25.1,24.8,23.1,22.1,3,Swiss Restaurant,Mediterranean Restaurant,Italian Restaurant,Museum,Grocery Store,French Restaurant,Tram Station,Wine Shop,Department Store,Dessert Shop
23,Weinegg,11'932,777,452,3'148,0.14,5'414,5'220,5'248,5'030,5'086,5'036,5'022,4'783,4'942,4'999,4'935,33.0,32.6,32.1,31.6,31.7,30.9,30.3,28.9,29.3,28.2,27.7,3,Swiss Restaurant,Mediterranean Restaurant,Italian Restaurant,Museum,Grocery Store,French Restaurant,Tram Station,Wine Shop,Department Store,Dessert Shop
3,City,32'084,1'924,3'006,1'245,2.41,797,829,830,810,805,791,783,799,779,835,853,31.7,30.0,30.5,29.5,30.2,29.0,29.9,31.4,30.6,32.2,33.3,3,Swiss Restaurant,Mediterranean Restaurant,Italian Restaurant,Museum,Grocery Store,French Restaurant,Tram Station,Wine Shop,Department Store,Dessert Shop


In [28]:
# Cluster 5
cluster5 = zurich_merged.loc[zurich_merged['labels'] == 4, zurich_merged.columns[[1] + list(range(5, zurich_merged.shape[1]))]]
cluster5.sort_values(by=['wp_pop_ratio'])

,quarter,employed,workplaces,wp_density,pop_density,wp_pop_ratio,pop2019,pop2018,pop2017,pop2016,pop2015,pop2014,pop2013,pop2012,pop2011,pop2010,pop2009,foreign2019,foreign2018,foreign2017,foreign2016,foreign2015,foreign2014,foreign2013,foreign2012,foreign2011,foreign2010,foreign2009,labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Leimbach,717,212,73,2'107,0.03,6'152,6'320,6'212,6'173,6'102,5'936,5'730,5'354,5'340,5'293,5'287,33.0,33.6,32.1,31.1,30.2,27.9,26.8,24.0,24.2,23.4,23.6,4,Moroccan Restaurant,Grocery Store,Tennis Court,French Restaurant,Food Court,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Yoga Studio


<a id='item4'></a>

## 4. Results

__Descriptive Results:__

Prior to the cluster analyses some descriptive results have to be discussed. The data from the Zurich statistical office contained _neighbourhood area, neighbourhood population, number of neighbourhood workplaces_ and _share of forein citizens._ Based on this data the (workplace density/population density)-ratio was computed to obtain a measure that reveals whether a specific neighbourhood is a "living neighbourhood" or a "working neighbourhood". The intuition is that neighbourhood with a low ratio are more likely to be "living neighbourhoods" (because there are relatively few workplaces in comparison to the neighbourhood population).

__Results of the Cluster Analyses:__

The k-means segmentation algorithm for Foursquare venues data resulted in five neighbourhood clusters for Zurich. Analyzing the most occuring venues in each cluster results in the following cluster classification:

__Cluster 1:__ _Urban Outskrits (red cluster)_
- Neighbourhoods in this cluster are characterized by many public transport venues, grocery stores and restaurants. Based on most occuring venues in this cluster is seems likely that neighbourhoods in this cluster are located at the interception between the urban area and the rural outskirts of Zurich. This interpretation is also supported by the fact that the ratio of workplace and population density is relatively low between 0.03 and 0.08, indicating that there are relatively few workplaces compared to the population living in these neighbourhoods. 

__Cluster 2:__ _Urban Downtown (purple cluster)
- Most frequent venues for neighbourhoods in the _Urban Downtown_ cluster are coffee shops, bars, restaurants and hotels. Hence, these neighbourhoods seem to be part of a lively urban area with lots of activities to do. A look at the (workplace density/population density)-ratio reveals that some of the _Urban Downtown_ areas are not really residential areas. When setting a ratio-threshold of ratio > 0.40 one can see that the neighbourhoods _Rathaus, Hochschule, Lindenhof_ and _city_ are mainly business districts. Hence, there neighbourhoods might not be the first choice for people who search for an apartment in Zurich. 

__Cluster 3:__ _Outskirt Cluster West (blue cluster)

__Cluster 4:__ _Outskirt Cluster North (green cluster)

__Cluster 5:__ _Outskirt Cluster South (orange cluster)

<a id='item5'></a>

## 5. Discussion

_will be updated in week 2_

<a id='item6'></a>

## 6. Conclusion

_will be updated in week 2_

## 7. References

- Mercer (2007). _2007 World-wide quality of living survey_. Retrieved from https://web.archive.org/web/20110812003533/http://www.mercer.com/referencecontent.htm?idContent=1173105
- Mercer (2008). _Mercer's 2008 Quality of living survey highlights_. Retrieved from https://mobilityexchange.mercer.com/Insights/quality-of-living-rankings